# 📚 RAG Wissensdatenbank mit Confluence Support

Dieses Notebook installiert und startet die RAG Wissensdatenbank mit öffentlichem Zugang über Cloudflare Tunnel.

## Voraussetzungen
- **OpenAI API Key** als Colab Secret (Name: `OPENAI_API_KEY`)

### Secret einrichten
1. Links auf das 🔑 Symbol klicken
2. "Neues Secret hinzufügen"
3. Name: `OPENAI_API_KEY`, Wert: Ihr API Key
4. "Notebook-Zugriff" aktivieren

### Confluence einrichten (nur Confluence Cloud!)
1. 👉 https://id.atlassian.com/manage-profile/security/api-tokens
2. Klicken Sie auf "Create API token"
3. Vergeben Sie einen Namen für den Token (z.B. "Wissensdatenbank")
4. Klicken Sie auf "Create"
5. Kopieren Sie das Token sofort (wird nur einmal angezeigt!)
6. Tragen Sie das Token in der App im Tab Confluence ein


## 1️⃣ Repository klonen

In [1]:
!git clone https://github.com/janschachtschabel/simple-document-rag.git
%cd simple-document-rag

Cloning into 'simple-document-rag'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 37 (delta 11), reused 30 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 73.01 KiB | 3.32 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/simple-document-rag


## 2️⃣ Abhängigkeiten installieren

In [2]:
!pip install fastapi
!pip install uvicorn
!pip install openai
!pip install sentence-transformers
!pip install chromadb
!pip install python-multipart
!pip install jinja2
!pip install aiofiles
!pip install pydantic
!pip install numpy
!pip install pandas
!pip install python-dotenv
!pip install langchain
!pip install langchain-openai
!pip install tiktoken
!pip install beautifulsoup4
!pip install langchain-community
!pip install atlassian-python-api
!pip install streamlit
!pip install rank-bm25
!pip install langgraph>=0.2.0
!pip install langchain-core>=0.3.0
!pip install markitdown[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/4

In [3]:
# Cloudflare Tunnel installieren
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!sudo dpkg -i cloudflared-linux-amd64.deb

Selecting previously unselected package cloudflared.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2026.1.2) ...
Setting up cloudflared (2026.1.2) ...
Processing triggers for man-db (2.10.2-1) ...


## 3️⃣ API Key laden und Modelle konfigurieren

In [4]:
import os
from google.colab import userdata

# API Key aus Colab Secrets laden
try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ OpenAI API Key geladen")
except Exception as e:
    print("❌ Secret 'OPENAI_API_KEY' nicht gefunden!")
    print("   🔑 Symbol links → Neues Secret → OPENAI_API_KEY")
    raise e

# Modelle (hier anpassen falls gewünscht)
OPENAI_MODEL = "gpt-4.1-mini"
EMBEDDING_MODEL = "text-embedding-ada-002"

os.environ["OPENAI_MODEL"] = OPENAI_MODEL
os.environ["EMBEDDING_MODEL"] = EMBEDDING_MODEL
os.environ["CHROMA_PERSIST_DIRECTORY"] = "./chroma_db"
os.environ["CHUNK_SIZE"] = "1000"
os.environ["CHUNK_OVERLAP"] = "200"
os.environ["TOP_K_RETRIEVAL"] = "5"

print(f"✅ LLM: {OPENAI_MODEL}")
print(f"✅ Embedding: {EMBEDDING_MODEL}")

✅ OpenAI API Key geladen
✅ LLM: gpt-4.1-mini
✅ Embedding: text-embedding-ada-002


## 4️⃣ FastAPI Server starten

In [5]:
import subprocess
import time
import requests

api_process = subprocess.Popen(
    ["python", "main.py"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

print("⏳ Starte API...")

# Retry-Loop: Warte bis API erreichbar ist (max 60 Sekunden)
api_ready = False
for i in range(12):  # 12 Versuche × 5 Sekunden = 60 Sekunden max
    time.sleep(5)
    try:
        r = requests.get("http://localhost:8000/health", timeout=3)
        if r.status_code == 200:
            api_ready = True
            break
    except:
        print(f"   Versuch {i+1}/12...")

if api_ready:
    print("✅ API läuft auf http://localhost:8000")
else:
    print("❌ API nicht erreichbar nach 60 Sekunden")
    print("   Führen Sie diese Zelle erneut aus oder prüfen Sie die Logs")

⏳ Starte API...
   Versuch 1/12...
   Versuch 2/12...
   Versuch 3/12...
   Versuch 4/12...
   Versuch 5/12...
   Versuch 6/12...
   Versuch 7/12...
✅ API läuft auf http://localhost:8000


## 5️⃣ Streamlit + Cloudflare Tunnel starten

Nach Ausführung erscheint eine **öffentliche URL**.

In [6]:
import subprocess
import re
import time

# Streamlit im Hintergrund starten
streamlit_process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
print("⏳ Starte Streamlit...")
time.sleep(5)

# Cloudflare Tunnel starten und URL extrahieren
def start_cloudflare_tunnel(port):
    print(f"🌐 Starte Cloudflare Tunnel für Port {port}...")
    process = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", f"http://localhost:{port}"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    for line in process.stderr:
        print(f"Cloudflare: {line.strip()}")
        if "trycloudflare.com" in line:
            match = re.search(r'https://[\w-]+\.trycloudflare\.com', line)
            if match:
                return match.group(0), process
    return None, process

tunnel_url, tunnel_process = start_cloudflare_tunnel(8501)

if tunnel_url:
    print("\n" + "=" * 60)
    print("🎉 RAG WISSENSDATENBANK IST ONLINE!")
    print("=" * 60)
    print(f"🔗 Public URL: {tunnel_url}")
    print("=" * 60)

⏳ Starte Streamlit...
🌐 Starte Cloudflare Tunnel für Port 8501...
Cloudflare: 2026-02-05T10:53:22Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
Cloudflare: 2026-02-05T10:53:22Z INF Requesting new quick Tunnel on trycloudflare.com...
Cloudflare: 2026-02-05T10:53:26Z INF +--------------------------------------------------------------------------------------------+
Cloudflare: 2026-02-05T10:53:26Z INF |  Your quick Tunnel has been created! Visit it at (it may tak

## 6️⃣ Status prüfen

In [9]:
import requests

print("📊 Status")
print("-" * 30)

try:
    r = requests.get("http://localhost:8000/health", timeout=5)
    print(f"✅ API: OK ({r.json().get('statistics', {}).get('total_documents', 0)} Dokumente)")
except:
    print("❌ API: Offline")

try:
    r = requests.get("http://localhost:8501", timeout=5)
    print("✅ Streamlit: OK")
except:
    print("❌ Streamlit: Offline")

if 'tunnel_process' in dir() and tunnel_process.poll() is None:
    print("✅ Tunnel: Aktiv")
else:
    print("❌ Tunnel: Inaktiv")

📊 Status
------------------------------
❌ API: Offline
❌ Streamlit: Offline
❌ Tunnel: Inaktiv


---
## 📝 Hinweise

- **Laufzeit**: Bis zu 12 Stunden (kostenlos)
- **Dokumente**: Gehen nach Sitzungsende verloren
- **Tunnel-URL**: Ändert sich bei jedem Neustart
- **Confluence**: In der App unter 🔷 Confluence konfigurieren